In [3]:
from Acesso import Login
from Moeda import Moeda
from Query import Query
from RemoverArquivo import Remover
from WebTransfer import Web
import telebot
from telebot import types
from datetime import datetime
from pathlib import Path
import shutil
import os
import pandas as pd
from decouple import config
from glob import glob
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from webdriver_manager.chrome import ChromeDriverManager
import pyttsx3

TOKEN=config('TOKEN')

bot=telebot.TeleBot(TOKEN,parse_mode='html')

s=Login()

sql=Query(s.usuario,s.senha,s.database,s.server)

commands_dict={

    '/foto':'Foto dos produtos',
    '/fichatecnica': 'Ficha técnica dos produtos',
    '/dadosprodutos': 'Dados dos produtos',
    '/carteira':'Carteira de cliente',
    '/vendas':'Resultado da empresa',
    '/realizado':'Resultado do dia atual',
    '/atendimento':'Atendimento X Carteira',
    '/estatistica':'Posição estatística comercial',
    '/meta': 'Meta mensais'

}

start={

    '/foto':'ValidacaoProduto',
    '/fichatecnica': 'ValidacaoProduto',
    '/dadosprodutos': 'ValidacaoProduto',
    '/carteira':'Carteira',
    '/vendas':'Vendas',
    '/realizado':'Realizado',
    '/atendimento':'Atendimento',
    '/estatistica':'Estatistica',
    '/meta':'Meta'

}

funcoes={

    '/foto':'Foto',
    '/fichatecnica': 'FichaTecnica',
    '/dadosprodutos': 'DadosProdutos',
    '/carteira':'Carteira',
    '/vendas':'Vendas',
    '/realizado':'Realizado',
    '/atendimento':'Atendimento',
    '/estatistica':'Estatistica',
    '/meta':'Meta'

}

espera=20

querys={

    'Produtos':

    """
        
    SELECT * FROM netfeira.vw_prod_dados
        
    """,

    'Vendedor':

    """
    
    SELECT LTRIM(RTRIM(vend.[ID Vendedor])) AS [ID Vendedor],vend.[Nome Resumido],
	sup.Equipe,vend.[Status do Vendedor],
    LTRIM(RTRIM(sup.[ID Sup])) AS [ID Sup],sup.Supervisor,[ID Gerente],Gerente
    FROM netfeira.vw_vendedor vend
    INNER JOIN netfeira.vw_supervisor sup ON vend.[ID Equipe]=sup.[ID Equipe]
    
    """,

    'Carteira':

    """

    SELECT cad.[ID Cliente],cad.CNPJ,cad.[Razão Social],cad.[Nome Fantasia],cad.Matriz,seg.Segmento,seg.Canal,
    cad.CEP,cad.Endereço,cad.Bairro,cad.Cidade,cad.UF,cad.Numero,cad.Complemento,cad.[Condição de Pagto],cad.Prazo,
    cad.Tabela,cad.DDD,cad.Contato,cad.[Última Compra],cad.Dias,
    cad.[ID Vendedor],cad.[Nome Resumido],cad.Equipe,cad.Supervisor,cad.Principal
    FROM netfeira.vw_carteira cad
    INNER JOIN netfeira.vw_segmento seg ON cad.[ID Segmento]=seg.[ID Segmento]
    
    """,

    'Mensal':

    """

    SELECT c.[ID Mês],c.Mês,c.[ID Vendedor],
    COALESCE(c.FATURADO,0) AS FATURADO,COALESCE(c.[DEVOLUÇÃO PARCIAL],0) AS [DEVOLUÇÃO PARCIAL],
    COALESCE(c.DEVOLUÇÃO,0) AS DEVOLUÇÃO,COALESCE(c.CANCELADO,0) AS CANCELADO,
    COALESCE(c.FATURADO,0)+COALESCE(c.[DEVOLUÇÃO PARCIAL],0)+COALESCE(c.DEVOLUÇÃO,0)+COALESCE(c.CANCELADO,0) AS [Total Líquido]
    FROM (

        SELECT *
        FROM (
            SELECT ca.[ID Mês],ca.Mês,
            LTRIM(RTRIM(v.[ID Vendedor])) AS [ID Vendedor]
            ,v.Situação,
            SUM(v.[Total Venda]) AS [Total Venda]
            FROM netfeira.vw_venda_estatico v
            INNER JOIN netfeira.vw_calend ca ON v.[Data de Faturamento]=ca.Data
            WHERE [Tipo de Operação]='VENDAS' AND YEAR(v.[Data de Faturamento])=YEAR(GETDATE())
            GROUP BY ca.[ID Mês],ca.Mês,v.[ID Vendedor],v.Situação

        )a
        PIVOT(SUM([Total Venda]) FOR Situação IN([FATURADO],[DEVOLUÇÃO],[DEVOLUÇÃO PARCIAL],[CANCELADO]))b

    )c
    
    """,

    'Realizado':

    """
    
    SELECT *
    FROM netfeira.vw_targetestatistico
    WHERE [Data de Emissão]=CONVERT(DATETIME,CAST(GETDATE() AS DATE),101) AND [Tipo de Operação]='VENDAS'
    
    """,

    'Meta':
    
    """
    
    SELECT m.[Mês Meta],c.Ano,c.Mês,m.[ID Vendedor],vend.[Nome Resumido],sup.Equipe,sup.Supervisor,
	m.[Meta R$]
	FROM netfeira.vw_metavend m
	INNER JOIN netfeira.vw_calend c ON m.[Mês Meta]=c.[Mês Meta]
	INNER JOIN netfeira.vw_vendedor vend ON m.[ID Vendedor]=vend.[ID Vendedor]
	INNER JOIN netfeira.vw_supervisor sup ON vend.[ID Equipe]=sup.[ID Equipe]
	WHERE c.Ano=YEAR(GETDATE())
    
    """,

    'Tabelas':


    """
    
    SELECT * FROM netfeira.vw_tab_vend
    
    """,

    'Estatistica':

    """

    SELECT * FROM netfeira.vw_targetestatistico
    WHERE YEAR([Data de Faturamento])=YEAR(GETDATE()) AND [Tipo de Operação]<>'OUTROS' AND [ID Situação] IN('FA','AB')
    
    """,

    'Calendario':

    """
    
    SELECT * FROM netfeira.vw_calend
    
    """,

    'Estatico':

    """
    
    SELECT * FROM netfeira.vw_venda_estatico
    WHERE YEAR([Data de Faturamento])=YEAR(GETDATE()) AND [Tipo de Operação]<>'OUTROS'
    
    """,

    'Uteis':

    """
    
    SELECT c.Ano,c.[ID Mês],c.Mês,COUNT(c.[Data]) AS [Dias Úteis],
    CASE WHEN [ID Mês]=MONTH(GETDATE()) THEN COUNT(c.[Data Trabalhada])-1 ELSE COUNT(c.[Data Trabalhada]) END AS [Trabalhado],
    COUNT(c.[Data])-
    CASE WHEN [ID Mês]=MONTH(GETDATE()) THEN COUNT(c.[Data Trabalhada])-1 ELSE COUNT(c.[Data Trabalhada]) END AS [Dias Restante]
    FROM netfeira.vw_calend c
    WHERE c.[Dia Útil]=1
    GROUP BY c.Ano,c.[ID Mês],c.Mês
    HAVING Ano=YEAR(GETDATE())
    ORDER BY [ID Mês]
    
    """

}

In [4]:
df=sql.GetDados(querys=querys,colunas=['Tabelas'])

c:\Users\eduardo\OneDrive - NETFEIRA PONTOCOM LTDA\Python\Homologação\Telegram BOT\Query\sql_query.py:37: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  temp_dict[c]=pd.read_sql(querys[c],self.conectando)


In [6]:
df['Tabelas'].columns

Index(['Empresa', 'Tabela', 'ID Vendedor', 'Nome Resumido', 'ID Sup',
       'Supervisor', 'ID Gerente', 'Gerente', 'SKU', 'Produto', 'Fabricante',
       'Categoria', 'Linha', 'Unid. VDA', 'Peso Liquido', 'Fator CX',
       'Peso Liquido Caixa', 'Preço VDA', 'Desc Máx', 'Valor C/Desc'],
      dtype='object')

In [9]:
df['Tabelas'][['Tabela','SKU', 'Produto', 'Fabricante',
       'Categoria', 'Linha', 'Unid. VDA', 'Peso Liquido', 'Fator CX',
       'Peso Liquido Caixa', 'Preço VDA', 'Desc Máx', 'Valor C/Desc']].drop_duplicates()

,Tabela,SKU,Produto,Fabricante,Categoria,Linha,Unid. VDA,Peso Liquido,Fator CX,Peso Liquido Caixa,Preço VDA,Desc Máx,Valor C/Desc
0,COML ESPERANÇA,3,POLPINHA DE ABACAXI (100G),DE MARCHI,POLPINHA,POLPINHA 100G,KG,1.00,6.0,6.00,14.93,0.0,14.93
5,COML ESPERANÇA,12,POLPINHA DE ACEROLA (100G),DE MARCHI,POLPINHA,POLPINHA 100G,KG,1.00,6.0,6.00,14.93,0.0,14.93
10,COML ESPERANÇA,35,POLPINHA DE AMORA (100G),DE MARCHI,POLPINHA,POLPINHA 100G,KG,1.00,6.0,6.00,21.93,0.0,21.93
15,COML ESPERANÇA,55,POLPINHA DE CAJA (100G),DE MARCHI,POLPINHA,POLPINHA 100G,KG,1.00,6.0,6.00,17.06,0.0,17.06
20,COML ESPERANÇA,57,POLPINHA DE CAJU (100G),DE MARCHI,POLPINHA,POLPINHA 100G,KG,1.00,6.0,6.00,22.90,0.0,22.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77156,TABELA OBA,9489,PAO AUST FORMA 350G PN10,BIMBO,ARTESANAL,ARTESANAL,UN,0.35,15.0,5.25,9.73,0.0,9.73
77160,TABELA OBA,9613,PAO MONACO ESCURO C/ GERGELIM CT CONG CXG 1440...,BIMBO,PÃES DE HAMBURGUER C/ GERGELIM,PÃO HB MONACO,UN,0.90,16.0,14.40,26.11,0.0,26.11
77164,TABELA OBA,9621,BOLO FORMIGUEIRO 330G PN10,BIMBO,BOLOS,BOLO PN10,UN,0.30,10.0,3.00,11.50,0.0,11.50
77168,TABELA OBA,9622,BOLO BANANA 330G PN10,BIMBO,BOLOS,BOLO PN10,UN,0.30,10.0,3.00,11.50,0.0,11.50


In [ ]:
for c in ['Estatistica','Meta','Estatico']:

    df[c]['ID Vendedor']=df[c]['ID Vendedor'].apply(RemoverEspaco)
    df[c]=df[c].loc[(df[c]['ID Vendedor'].isin(codigos))&(df[c]['Mês']==val)]
                
    pass